In [ ]:
import sys
sys.path.append('./helpers')
import pyVoxelFEM
from ipopt_helpers import initializeTensorProductSimulator, problemObjectWrapper
from debug_helpers import numericalDerivative, plotConvergence

import numpy as np
import matplotlib.pyplot as plt

# Numerical derivatives
Use numerical differentiation to check the analytical form of the derivatives implemented in the code. Results are shown both in terms of numerical value and with error plots (numerical scheme is centered differences, so the error is expected to decrease as $h^2$)

### TO problem derivatives
Test **objective** and **constraints** derivatives

In [ ]:
# 2D or 3D?
DIM = 2

# Parameters
MATERIAL_PATH = '../examples/materials/B9Creator.material'
if DIM == 2:
    BC_PATH = '../examples/bcs/cantilever_flexion_E.bc'
    orderFEM = [1, 1]
    domainCorners = [[0, 0], [2, 1]]
    gridDimensions = [16, 8]
elif DIM == 3:
    BC_PATH = '../examples/bcs/3D/cantilever_flexion_E.bc'
    orderFEM = [1, 1, 1]
    domainCorners = [[0, 0, 0], [2, 1, 1]]
    gridDimensions = [8, 4, 4]
    
elementsNumber = int(np.prod(gridDimensions))
E0 = 1
Emin = 1e-9
SIMPExponent = 3
maxVolume = 0.6
constraints = [
    pyVoxelFEM.TotalVolumeConstraint(maxVolume)
]
smoothingRadius = 0
filters = [
    pyVoxelFEM.SmoothingFilter(),
    pyVoxelFEM.ProjectionFilter(),
    pyVoxelFEM.LangelaarFilter()
]
uniformDensity = maxVolume

# Initialize TO problem
tps = initializeTensorProductSimulator(
    orderFEM, domainCorners, gridDimensions, uniformDensity, E0, Emin, SIMPExponent, MATERIAL_PATH, BC_PATH
)
objective = pyVoxelFEM.ComplianceObjective(tps)
prob = pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, filters)
x0 = tps.getDensities()

# Wrap problem into ipopt interface to make sure of testing the actual object used by the optimizer
ipoptProblemObj = problemObjectWrapper(prob)

In [ ]:
# Check constraints jacobian
x = tps.getDensities()
increments = np.linspace(1e-4, 1e-2, 10)
direction = np.zeros(x.size)
variableIndex = 50
direction[variableIndex] += 1

print("Constraints jacobian, derivative w.r.t. " + str(variableIndex) + "-th density:")
print("Analytical:", ipoptProblemObj.jacobian(x).dot(direction))
print("Numerical: ", numericalDerivative(ipoptProblemObj.constraints, x, increments[0], direction))
title = "Constraint jacobian: numerical derivative check"
plotConvergence(ipoptProblemObj.constraints, x, increments, direction, ipoptProblemObj.jacobian(x).dot(direction), title)

In [ ]:
# Check objective gradient
x = tps.getDensities()
increments = np.linspace(1e-4, 1e-2, 10)
direction = np.zeros(x.shape[0])
variableIndex = 50
direction[variableIndex] += 1

print("Objective gradient, derivative w.r.t. " + str(variableIndex) + "-th density:")
print("Analytical:", ipoptProblemObj.gradient(x).dot(direction))
print("Numerical: ", numericalDerivative(ipoptProblemObj.objective, x, increments[0], direction))
title = "Objective gradient: numerical derivative check"
plotConvergence(ipoptProblemObj.objective, x, increments, direction, ipoptProblemObj.gradient(x).dot(direction), title)